## Aircraft Predictive Maintenance

Dataset:
- https://www.kaggle.com/datasets/maternusherold/pred-maintanance-data?resource=download
- Copied to /dataset

Include:
- Data preparation and data cleaning
- EDA, feature importance analysis
- Model selection process and parameter tuning


In [ ]:
# Download Dataset